# Titanic machine learning homework

we import numpy, pandas, matplotlib, os and sklearn tools for processing data

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import os 
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

The data was downloaded from the kaggle website and read via pandas read_csv function 

In [2]:
# import datasets
train_dataset = pd.read_csv('/home/ravi/coding/ML/datasets/titanic/input/train.csv')
test_dataset = pd.read_csv('/home/ravi/coding/ML/datasets/titanic/input/test.csv')

In [12]:
#learn about the training dataset
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


The Ttrain only contains 3 attributes sex, age, and pclass. Therefore, all other attributes were dropped. The cat.codes method was used to get a numerical value of the sex column. The data was then transformed using the minmaxscaler to get all the values between 0 and 1. The scaler also changes the datadype to arrays,and we change it back to dataframe by using DataFrame function. We get rid of the NaN values in the training data by replacing them with the median values respectively. 

In [4]:
##prepare the dataset for training 

# drop all other attributes
Ttrain = train_dataset.drop(['PassengerId','Survived','Name','SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
Tlabels= train_dataset["Survived"].copy()

# minmaxscaler
Ttrain['Sex']=Ttrain.Sex.astype("category").cat.codes
min_max_scaler = preprocessing.MinMaxScaler()
Ttrain= min_max_scaler.fit_transform(Ttrain)

# dataframe conversion 
Ttrain = pd.DataFrame(Ttrain, columns=['Pclass','Sex','Age'])

# using median values for NaN 
median = Ttrain['Age'].median()
Ttrain['Age']=Ttrain['Age'].fillna(median)

/home/ravi/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


The decision tree model is used to train and predict. 

In [5]:
#Decision tree model, train, and predict 
tree = DecisionTreeRegressor()
tree.fit(Ttrain,Tlabels)
Tpredict = tree.predict(Ttrain)

In [6]:
#see score function 
def display_scores(scores):
    print("scores: ", scores)
    print("Mean: ", scores.mean())
    print("Std Dev: ", scores.std())

The mean squared error is used to see the output scores. The cross_val_score is used to do cross validation on the training 

In [7]:
#Check the status and scores 
tree_mse = mean_squared_error(Tpredict, Tlabels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

scores = cross_val_score(tree, Ttrain, Tlabels, scoring ='neg_mean_squared_error',cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

scores:  [0.43823768 0.41861711 0.42486024 0.40778203 0.37458807 0.40416767
 0.39556198 0.43077483 0.33274161 0.38934782]
Mean:  0.4016679045803088
Std Dev:  0.029540769432071363


The data for testing is prepared the same way as it was done in the training process. 

In [8]:
##prepare the dataset for prediction 

# drop all other attributes
Ttest = test_dataset.drop(['PassengerId','Name','SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

# minmaxscaler
Ttest['Sex']=Ttest.Sex.astype("category").cat.codes

Ttest= min_max_scaler.fit_transform(Ttest)

# dataframe conversion 
Ttest = pd.DataFrame(Ttest, columns=['Pclass','Sex','Age'])

# using median values for NaN 
median = Ttest['Age'].median()
Ttest['Age']=Ttest['Age'].fillna(median)

/home/ravi/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


The variable 'a' holds the prediction data for the test data. All the float values are converted to int when changing from array to pandas dataframe. any value 0.5 and above was changed to 1 and any value below 0.5 was changed to 0, which represents survived and dead respectively. The variable bigdata is a pandas dataframe which combines the passengerID column and the Survived column into one dataframe, which is written to one csv file for submittion. 

In [16]:
a = tree.predict(Ttest)
a[a >= 0.5] = 1
a[a < 0.5] = 0
ab = pd.DataFrame(data=a,columns=['Survived']).astype(int)
b = test_dataset['PassengerId']
bb = pd.DataFrame(data=b,columns=['PassengerId']).astype(int)
bigdata = pd.concat([bb, ab], axis=1,join='inner' )
bigdata

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [18]:
bigdata.to_csv('save2.csv', encoding='utf-8', index=False)